In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from huggingface_hub import login
login(token="YOUR_HUGGING_FACE_TOKEN")

In [ ]:
import sys
import os # Good for constructing paths robustly

# --- 1. Add your dataset path to sys.path ---
# The actual path name Kaggle uses might vary slightly (e.g., underscores vs hyphens)
# Check the "Input" section of your Kaggle notebook's file explorer to confirm the exact path.
dataset_name_on_kaggle = "rope12" # The name you gave your dataset
dataset_path = os.path.join("/kaggle/input", dataset_name_on_kaggle)

if dataset_path not in sys.path:
    sys.path.append(dataset_path)
    print(f"Added {dataset_path} to sys.path")
else:
    print(f"{dataset_path} already in sys.path")

In [ ]:
from custom_arch import LlamaAttentionWithCustomRope

In [ ]:
# from custom_arch import LlamaAttentionWithCustomRope
import torch
from transformers.models.llama import modeling_llama
from transformers import AutoModelForCausalLM, AutoTokenizer

orig_llama_LlamaAttention = modeling_llama.LlamaAttention

modeling_llama.LlamaAttention = LlamaAttentionWithCustomRope

model_id = "meta-llama/Llama-3.2-3B-Instruct" # Or your specific Llama model
tokenizer = AutoTokenizer.from_pretrained(model_id)

print(f"Loading model: {model_id}...")
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16, # Or your preferred dtype
    device_map="auto"           # Automatically use GPU if available
)
print("Model loaded.")

In [ ]:
prompt = "What is the capital of France?"
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

print("Generating text with the (potentially patched) model...")
with torch.no_grad():
    outputs = model.generate(**inputs, max_new_tokens=20)

print("Generated output:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))